# Chapter 4 구조적 API 개요

구조적 API에는 세 가지 분산 컬렉션 API가 있다
* Dataset
* DataFrame
* SQL 테이블과 뷰

**배치**와 **스트리밍** 처리에서 구조적 API를 사용할 수 있다. 구조적 API를 활용하여 배치 작업을 스트리밍 작업으로 손쉽게 변경할 수 있다(반대의 경우도 마찬가지이다.)

구조적 API는 데이터 흐름을 정의하는 기본 추상화 개념이다. 
* 타입형/비타입형 API의 개념과 차이점
* 핵심 용어
* 스파크가 구조적 API의 데이터 흐름을 해석하고 클러스터에서 실행하는 방식

## 4.1 DataFrame과 Dataset

DataFrame과 Dataset은 잘 정의된 로우와 컬럼을 가지는 분산 테이블 형태의 컬렉션이다.<br>
각 컬럼은 다른 컬럼과 동일한 수의 로우를 가져야 한다. 그리고 컬렉션의 모든 로우는 같은 데이터 타입 정보를 가지고 있어야 한다.

DataFramerhk Dataset은 결과를 생성하기 위해 어떤 데이터에 어떤 연산을 적용해야 하는지 정의하는 지연 연산의 실행 계획이며, 불변성을 가진다. DataFrame에 액션을 호출하면 스파크는 트랜스포메이션을 실제로 실행하고 결과를 반환한다. 이 과정은 사용자가 원하는 결과를 얻기 위해 로우와 컬럼을 처리하는 방법에 대한 계획을 나타낸다.

## 4.2 스키마

* 스키마 : 분산 컬렉션에 저장할 데이터 타입을 정의하는 방법이다.

## 4.3 스파크의 구조적 데이터 타입 개요

스파크는 실행 계획 수립과 처리에 사용하는 자체 데이터 타입 정보를 가지고 있는 **카탈리스트** 엔진을 사용한다.<br>
카탈리스트 엔진은 다양한 실행 최적화 기능을 제공한다. 스파크는 자체 데이터 타입을 지원하는 여러 언어 API와 직접 매핑되며, 각 언어에 대한 매핑 테이블을 가지고 있다.

In [ ]:
df = spark.range(500).toDF("number")
df.select(df["number"] + 10)

카탈리스트 엔진에서 스파크의 데이터 타입으로 변환해 명령을 처리

### 4.3.1 DataFrame과 Dataset 비교

본질적으로 구조적 API에는 '비타입형' DataFrame과 '타입형'인 Dataset이 있다.

In [10]:
from pyspark.sql.types import *

b  = ByteType()
b

ByteType

* 스파크 데이터 타입을 파이썬에서 사용

## 4.4 구조적 API의 실행 과정

1. DataFrame/Dataset/SQL을 이용해 코드를 작성
2. 정장적인 코드라면 스파크가 **논리적 실행 계획**으로 변환
3. 스파크는 **논리적 실행 계획**을 **물리적 실행** 계획으로 변환하며 그 과정에서 추가적인 최적화를 할 수 있는지 확인
4. 스파크는 클러스터에서 **물리적 실행 계획**(RDD 처리)을 실행

![](./img/IMG_B6755434DFC6-1.jpeg)

### 4.4.1 논리적 실행 계획

![](./img/IMG_C4CC77A14B4B-1.jpeg)

* 사용자 코드를 논리적 실행 계획으로 변환
* 논리적 실행 계획 단계이서는 추상적 트랜스포메이션만 표현(드라이버나 익스큐터 정보 고려 X)
* 사용자 코드 -> **검증 전 논리적 실행 계획** 변환
* 스파크 분석기는 컬럼과 테이블을 검증하기 위해 카탈로그, DataFrame정보를 활용
* 테이블과 컬럼에 대한 검증 결과는 카탈리스트 옵티마이저로 전달
* 카탈리스트 옵티마이저는 조건절 푸시 다운이나 선택절 구문을 이용해 논리적 실행 계획을 최적화라는 규칙의 모음
* 도메인에 최적호괸 규칙을 적용가능해게 커스텀 가능

### 4.4.2 물리적 실행 계획

![](./img/IMG_37EED158612E-1.jpeg)

**물리적 실행 계획은 논리적 실행 계획을 클러스터 환경에서 실행하는 방법을 정의한다**

* 다양한 물리적 실행 전략을 생성
* 비용 모델을 이용해서 비교 후 최적의 전략을 선택
* 조인 연산 수행에 필요한 비용을 계산하고 비교
* 물리적 실행 계획은 일련의 RDD와 트렌스포메이션으로 변환
* 스파크는 DataFrame, Dataset, SQL로 정의된 퀴리를 RDD 트렌스모메이션으로 컴파일

**스파크를 '컴파일러'라고 부르기도 한다.**

### 4.4.3 실행

스파크는 물리적 실행 계획을 선정한 다음 저수즌 프로그래밍 인터페이스인 RDD를 대상으로 모든 코드를 실행한다.
스파크는 런타인에 전체태스크나 스테이지를 제거할 수 있는 자바 바이트 코드를 생성해 추가적인 최적화를 수행. 마지박으로 스파크는 처리 결과를 사용자에거 반환한다.